In [1]:
#################################### formats the pgp survey for machine learning ################################

In [2]:
import numpy as np

In [6]:
# load the pgp names as numpy array
names = np.load("C:\\Users\\j\\Desktop\\tiling_rep\\names.npy")

# load pgp array to a list
names_lst = np.ndarray.tolist(names)

# load the pgp genomes as numpy array
pgp = np.load("C:\\Users\\j\\Desktop\\tiling_rep\\hiq-pgp.npy")

# creates an empty blood csv file to be used later
out = open("bnry_blood.csv", "w")
out.close()

In [7]:
# formats a single tile by cleaning the name to make it more readable

def format(npint):
    s = ""
    string = str(npint)
    for char in range(len(string)):
        if string[char] == "-":
            break
        elif string[char] != "b" and string[char] != "'":
            s += string[char]
        else:
            pass
    return s



# runs formatting on all names

def formatall():

    tiled_partic = [] 
    for i in range(len(names_lst)):
        tiled_partic.append(format(names_lst[i]))
        
    return tiled_partic



tiled_names = formatall()

print(len(tiled_names))

218


In [8]:
# opens the PGP survey and dumps the participant list filtering non responses and duplicates

import csv 
with open("pgp_blood18.csv", newline='') as blood:
    reader = csv.DictReader(blood)
    blood_names = []
    for row in reader:
        if row["blood"] == "":
            pass
        elif row["blood"] == "Don't know":
            pass
        else:
            blood_names.append(row["name"])
        

def removedoubles(seq):
    seen = set()
    seen_add = seen.add
    return [x for x in seq if not (x in seen or seen_add(x))]


# takes out names that occur both in pgp survey and tiled genome list
blood_names = [x for x in blood_names if x in tiled_names]

blood_names_unfil = blood_names
blood_names = removedoubles(blood_names) 

print("Blood Survey Participants:",len(blood_names))

Blood Survey Participants: 65


In [9]:
# finds repeat values and puts them in a list

seen = set()
repeats = []
for x in blood_names_unfil:
    if x in seen:
        repeats.append(x)
    else:
        seen.add(x)

In [10]:
# makes a list of names that are in both the PGP survey name list and the tiled genomes list

print(len(repeats))


4


In [22]:
# takes in blood values and converts to a binary 1 or 0 format

# list of blood types for readability
blood_type_lst = []

# enter the survey condition here
condition = "+"


def to_bnry(field):
    if condition in field:
        return 1
    else:
        return 0
 

# creates a list of numbers 1 or 0 that represent whether the condition in to_bnry was met or not
bnry_list = []
seen_partic = []
with open('pgp_blood18.csv', newline='') as inp:
    reader = csv.DictReader(inp) 
    blood = ""
    for row in reader:
        if row['name'] in blood_names:
            blood = row['blood']   
            if row['name'] in seen_partic:
                pass
            else:
                bnry_list.append(to_bnry(blood))
                blood_type_lst.append(blood)
            seen_partic.append(row['name'])
                  
        else:
            pass
                
bnry = bnry_list

print(bnry)

print("rows:", len(bnry))

matches = [x for x in bnry if x == 1]

print(condition, " :", len(matches))

print("not",condition, " :", len(bnry) -len(matches) )


[1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1]
rows: 65
+  : 55
not +  : 10


In [23]:
# keeps only the particpants of the disease survey from the pgp array and
# discards the rest

filtered_pgp = []
name_list = []
for i in range(len(pgp)):
    if format(names[i]) in blood_names and format(names[i]) not in name_list:
        filtered_pgp.append(pgp[i])
        name_list.append(format(names[i]))
    else:
        pass
    
filtered_pgp = np.asarray(filtered_pgp)

filtered_pgp.shape

(65, 2469062)

In [24]:
# writes the binary lists to a csv file

import csv
l = list(map(str, bnry))

output_name = "bnry_blood_" + str(condition) + ".csv"
out = open(output_name, "w")
for row in l:
    for column in row:
        out.write(column)
    out.write("\n")
out.close()

In [26]:
# writes the list of blood types to a csv file

import csv
l = blood_type_lst

output_name = "bnry_blood_type_lst.csv"
out = open(output_name, "w")
for row in l:
    for column in row:
        out.write(column)
    out.write("\n")
out.close()

In [27]:
filtered_pgp.dump("pgp_filtered_blood.pkl")